# Домашнє завдання: ETL-пайплайни для аналітиків даних

Це ДЗ передбачене під виконання на локальній машині. Виконання з Google Colab буде суттєво ускладнене.

## Підготовка
1. Переконайтесь, що у вас встановлены необхідні бібліотеки:
   ```bash
   pip install sqlalchemy pymysql pandas matplotlib seaborn python-dotenv
   ```

2. Створіть файл `.env` з параметрами підключення до бази даних classicmodels. Базу даних ви можете отримати через

  - docker-контейнер згідно існтрукції в [документі](https://www.notion.so/hannapylieva/Docker-1eb94835849480c9b2e7f5dc22ee4df9), також відео інструкції присутні на платформі - уроки "MySQL бази, клієнт для роботи з БД, Docker і ChatGPT для запитів" та "Як встановити Docker для роботи з базами даних без терміналу"
  - або встановивши локально цю БД - для цього перегляньте урок "Опціонально. Встановлення MySQL та  БД Сlassicmodels локально".
  
  Приклад `.env` файлу ми створювали в лекції. Ось його обовʼязкове наповнення:
    ```
    DB_HOST=your_host
    DB_PORT=3306 або 3307 - той, який Ви налаштували
    DB_USER=your_username
    DB_PASSWORD=your_password
    DB_NAME=classicmodels
    ```
  Якщо ви створили цей файл під час перегляду лекції - **новий створювати не треба**. Замініть лише назву БД, або пропишіть назву в коді створення підключення (замість отримання назви цільової БД зі змінних оточення). Але переконайтесь, що до `.env` файл лежить в тій самій папці, що і цей ноутбук.

  **УВАГА!** НЕ копіюйте скрит для **створення** `.env` файлу. В лекції він наводиться для прикладу. І давалось пояснення, що в реальних проєктах ми НІКОЛИ не пишемо доступи до бази в коді. Копіювання скрипта для створення `.env` файлу сюди в ДЗ буде вважатись грубою помилкою і ми зніматимемо бали.

3. Налаштуйте підключення через SQLAlchemy до БД за прикладом в лекції.

Рекомендую вивести (відобразити) змінну engine після створення. Вона має бути не None! Якщо None - значить у Вас не підтягнулись налаштування з .env файла.

Ви також можете налаштувати параметри підключення до БД без .env файла, просто прописавши текстом в відповідних місцях. Це - не рекомендований підхід.

### Завдання 1: Створення таблиці курсів валют та API інтеграція (2 бали)

**Повторіть процедуру з лекції:** створіть таблицю для курсів валют, але вже в цій базі даних. Результатом має бути нова таблиця з курсами валют USD, EUR, UAH в БД (можна завантажити більше валют). Продемонструйте, що таблиця була додана, використовуючи SELECT.

Тобто тут ви можете прямо скопіювати код з лекції, внести необхідні зміни і запустити. Головне - отримати таблицю в БД classicmodels.

In [6]:
import os
import pandas as pd
import requests
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

In [7]:
load_dotenv(r"C:\Users\a.nemogushcha\OneDrive - ТОВ Смарт Дистрибюшн\Documents\SQLite\mysqlpass.env")

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [10]:
df = pd.read_sql("SHOW TABLES;", engine)
df

,Tables_in_classicmodels
0,currency_rates
1,customers
2,employees
3,offices
4,orderdetails
5,orders
6,payments
7,productlines
8,products


In [11]:
def create_currency_table(engine):
    create_sql = """
    CREATE TABLE IF NOT EXISTS currency_rates (
        id INT AUTO_INCREMENT PRIMARY KEY,
        currency_code CHAR(3) NOT NULL,
        rate_to_usd DECIMAL(18,6) NOT NULL,
        date DATE NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
        UNIQUE KEY uq_currency_date (currency_code, date)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    with engine.connect() as conn:
        conn.execute(text(create_sql))
        conn.commit()

create_currency_table(engine)


In [34]:
import requests
from datetime import date

API_KEY = 

def fetch_exchange_rates():
    url = "https://api.apilayer.com/exchangerates_data/latest"
    keep = ["EUR", "GBP", "UAH", "PLN", "JPY"]
    params = {"symbols": ",".join(keep + ["USD"])} 
    headers = {"apikey": API_KEY}

    try:
        r = requests.get(url, params=params, headers=headers, timeout=20)
        r.raise_for_status()
        js = r.json()

        if js.get("success") is False:
            raise RuntimeError(js.get("error", {}))

        rates_all = js.get("rates", {})
        usd = float(rates_all["USD"]) 
        rate_date = js.get("date", str(date.today()))

        rates = {c: float(rates_all[c]) / usd for c in keep if c in rates_all}
        print(f"✅ Отримали {len(rates)} валют на {rate_date}")
        return rates, rate_date

    except Exception as e:
        print("❌ Помилка API:", e)
        return None, None


In [39]:
create_currency_table(engine)

print(" Отримуємо курси валют...")
rates, rate_date = fetch_exchange_rates()


if rates:
    for cur, rt in rates.items():
        print(f"1 USD = {rt:.4f} {cur}")

    if save_exchange_rates(engine, rates, rate_date):
        df = pd.read_sql("""
            SELECT currency_code, rate_to_usd, rate_date, created_at, updated_at
            FROM currency_rates
            ORDER BY rate_date DESC, currency_code
            LIMIT 10
        """, engine)
        display(df)
else:
    print(" Немає даних для збереження")


 Отримуємо курси валют...
✅ Отримали 5 валют на 2025-08-10
1 USD = 0.8585 EUR
1 USD = 0.7446 GBP
1 USD = 41.3144 UAH
1 USD = 3.6477 PLN
1 USD = 147.6415 JPY
✅ Збережено 5 рядків на 2025-08-10


,currency_code,rate_to_usd,rate_date,created_at,updated_at
0,EUR,0.858504,2025-08-10,2025-08-10 15:39:24,2025-08-10 16:04:02
1,GBP,0.744574,2025-08-10,2025-08-10 15:39:24,2025-08-10 16:04:02
2,JPY,147.641478,2025-08-10,2025-08-10 15:39:24,2025-08-10 16:04:02
3,PLN,3.647740,2025-08-10,2025-08-10 15:39:24,2025-08-10 16:04:02
4,UAH,41.314443,2025-08-10,2025-08-10 15:39:24,2025-08-10 16:04:02


# Завдання 2: Створення простого ETL пайплайну (7 балів)

В цьому завданні ми створимо повноцінний ETL процес для аналізу продажів ClassicModels.

Завдання обʼємне і оцінюється відповідно. Ви можете пропустити обчислення якихось з метрик, якщо відчуєте, що вже немає сил робити це завдання. Бал буде виставлено виходячи з виконаного обʼєму та його правильності.

## Що саме треба зробити:

### Extract (Витягування даних):
На цьому етапі треба витягнути дані з БД в pandas.DataFrame для подальшої обробки.
Які дані нам потрібні (кожен пункт - в окремий фрейм даних):
1. **дані про виконані замовлення за 2004 рік** - з'єднати таблиці orders, orderdetails, products, customers
2. **дані про продукти** - назви, категорії, ціни
3. **дані про курси валют** - використати дані з попереднього завдання

### Transform (Обробка даних):

#### 2.1 Додати розрахункові колонки до основної таблиці:
Додайте до DataFrame з продажами такі нові колонки:

- **`profit_per_item`** - прибуток з одного товару (використайте колонки: `priceEach` - `buyPrice`)
- **`total_profit`** - загальний прибуток з товарної позиції (використайте колонки: `profit_per_item` × `quantityOrdered`)
- **`total_amount_eur`** - сума в євро (використайте колонки: `total_amount` / `eur_rate`)

#### 2.2 Створити аналітичну таблицю по країнах (ТОП-5):
Згрупуйте дані по колонці **`country`** та обчисліть для кожної країни:

**Метрики для розрахунку:**
- **Кількість унікальних замовлень** - унікальні значення колонки `orderNumber`
- **Загальний дохід** - сума колонки `total_amount`
- **Загальний прибуток** - сума колонки `total_profit`
- **Кількість проданих товарів** - сума колонки `quantityOrdered`
- **Маржа прибутку (%)** - (`загальний прибуток` / `загальний дохід`) × 100

**Результат:** Таблиця з 5 найприбутковіших країн, відсортована за загальним доходом (від більшого до меншого).

#### 2.3 Створити аналітичну таблицю по продуктових лініях:
Згрупуйте дані по колонці **`productLine`** та обчисліть ті ж метрики:

**Метрики для розрахунку:**
- **Кількість унікальних замовлень** - унікальні значення колонки `orderNumber`
- **Загальний дохід** - сума колонки `total_amount`
- **Загальний прибуток** - сума колонки `total_profit`
- **Кількість проданих товарів** - сума колонки `quantityOrdered`
- **Маржа прибутку (%)** - (`загальний прибуток` / `загальний дохід`) × 100

**Результат:** Таблиця з усіма продуктовими лініями, відсортована за загальним доходом.

#### 2.4 Створити підсумкову інформацію (Executive Summary):
Розрахуйте загальні показники бізнесу за 2004 рік:

**Фінансові показники:**
- **Загальний дохід в доларах** - сума всієї колонки `total_amount`
- **Загальний дохід в євро** - сума всієї колонки `total_amount_eur`
- **Загальний прибуток в доларах** - сума всієї колонки `total_profit`
- **Загальна маржа прибутку (%)** - (`загальний прибуток` / `загальний дохід`) × 100
- **Середній розмір замовлення** - середнє значення колонки `total_amount`

**Операційні показники:**
- **Кількість унікальних замовлень** - унікальні значення колонки `orderNumber`
- **Кількість унікальних клієнтів** - унікальні значення колонки `customerName`
- **Період даних** - мінімальна та максимальна дата з колонки `orderDate`

**Топ показники:**
- **Найприбутковіша країна** - перший рядок з таблиці країн (колонка `country`)
- **Найприбутковіша продуктова лінія** - перший рядок з таблиці продуктів (колонка `productLine`)

### Load (Збереження результатів):
В цій частині ми зберігаємо результати наших обчислень.
Використайте приклади коду з лекцій та адаптуйте його під цей ETL процес.
Що Вам потрібно створити:

#### 3.1 Excel файл з трьома вкладками:
- **"Summary"** - підсумкова інформація у вигляді таблиці "Показник - Значення"
- **"Top_Countries"** - аналітика по топ-5 країнах
- **"Product_Lines"** - аналітика по всіх продуктових лініях

#### 3.2 Візуалізація:
- Створіть стовпчикову діаграму топ-5 країн за доходом.
- Створіть pie chart з відсотковим розподілом доходу в USD по продуктових лінійках.

## РЕКОМЕНДАЦІЇ ДО ВИКОНАННЯ:

### Покрокова стратегія виконання:
1. Спочатку протестуйте Extract просто в Jupyter notebook (без фукнції) - переконайтеся що SQL запит працює і повертає дані за 2004 рік
2. Потім протестуйте кожен Transform окремо - виведіть проміжні результати
3. Нарешті протестуйте Load - перевірте що файли створюються правильно  
4. Тільки після цього обгортайте все в функцію

### Як перевірити що все працює:
- Виводьте на екран, який етап зараз відбувається
- Виведіть кількість записів після кожного кроку
- Покажіть перші 5 рядків кожної аналітичної таблиці
- Перевірте що дати належать 2004 року
- Переконайтеся що маржа прибутку в розумних межах (0-50%)

In [46]:
sql_extract = text("""
SELECT
    o.orderNumber,
    o.orderDate,
    c.customerNumber,
    c.country,
    p.productLine,
    od.productCode,
    od.quantityOrdered,
    od.priceEach,
    pr.buyPrice
FROM orders o
JOIN orderdetails od ON o.orderNumber = od.orderNumber
JOIN products pr       ON od.productCode = pr.productCode
JOIN customers c       ON o.customerNumber = c.customerNumber
JOIN productlines p    ON pr.productLine = p.productLine
WHERE YEAR(o.orderDate) = 2004
""")

df_raw = pd.read_sql(sql_extract, con=engine, parse_dates=["orderDate"])
print("EXTRACT: рядків =", len(df_raw))
display(df_raw.head())


EXTRACT: рядків = 1421


,orderNumber,orderDate,customerNumber,country,productLine,productCode,quantityOrdered,priceEach,buyPrice
0,10208,2004-01-02,146,France,Classic Cars,S12_1108,46,176.63,95.59
1,10208,2004-01-02,146,France,Classic Cars,S12_3148,26,128.42,89.14
2,10208,2004-01-02,146,France,Classic Cars,S12_3891,20,152.26,83.05
3,10208,2004-01-02,146,France,Vintage Cars,S18_3140,24,117.47,68.30
4,10208,2004-01-02,146,France,Trains,S18_3259,48,96.81,67.56


In [47]:
df = df_raw.copy()

df["profit_per_item"] = df["priceEach"] - df["buyPrice"]
df["total_profit"] = df["profit_per_item"] * df["quantityOrdered"]
df["total_amount"] = df["priceEach"] * df["quantityOrdered"]

print("TRANSFORM 2.1: додали колонки")
display(df[["orderNumber","productCode","quantityOrdered","priceEach","buyPrice","profit_per_item","total_profit","total_amount"]].head())


TRANSFORM 2.1: додали колонки


,orderNumber,productCode,quantityOrdered,priceEach,buyPrice,profit_per_item,total_profit,total_amount
0,10208,S12_1108,46,176.63,95.59,81.04,3727.84,8124.98
1,10208,S12_3148,26,128.42,89.14,39.28,1021.28,3338.92
2,10208,S12_3891,20,152.26,83.05,69.21,1384.20,3045.20
3,10208,S18_3140,24,117.47,68.30,49.17,1180.08,2819.28
4,10208,S18_3259,48,96.81,67.56,29.25,1404.00,4646.88


In [48]:
gb_country = df.groupby("country").agg(
    orders_cnt=("orderNumber", "nunique"),
    total_amount=("total_amount", "sum"),
    total_profit=("total_profit", "sum"),
    quantity=("quantityOrdered", "sum"),
)
gb_country["avg_order_value"] = gb_country["total_amount"] / gb_country["orders_cnt"]
gb_country["profit_margin_pct"] = (gb_country["total_profit"] / gb_country["total_amount"] * 100)

top_countries = gb_country.sort_values("total_amount", ascending=False).head(5).reset_index()

print("TRANSFORM 2.2: топ-5 країн за доходом")
display(top_countries)


TRANSFORM 2.2: топ-5 країн за доходом


,country,orders_cnt,total_amount,total_profit,quantity,avg_order_value,profit_margin_pct
0,USA,53,1526499.65,614370.08,16719,28801.880189,40.246985
1,France,19,506660.01,211528.15,5632,26666.316316,41.749525
2,Spain,14,439881.84,175328.56,4962,31420.131429,39.858104
3,UK,7,238193.93,93425.03,2778,34027.704286,39.222255
4,New Zealand,6,233362.27,94390.14,2537,38893.711667,40.447901


In [49]:
gb_pl = df.groupby("productLine").agg(
    orders_cnt=("orderNumber", "nunique"),
    total_amount=("total_amount", "sum"),
    total_profit=("total_profit", "sum"),
    quantity=("quantityOrdered", "sum"),
)
gb_pl["avg_order_value"] = gb_pl["total_amount"] / gb_pl["orders_cnt"]
gb_pl["profit_margin_pct"] = (gb_pl["total_profit"] / gb_pl["total_amount"] * 100)

product_lines = gb_pl.sort_values("total_amount", ascending=False).reset_index()

print("TRANSFORM 2.3: аналітика по продуктових лініях (відсортовано)")
display(product_lines.head(10))


TRANSFORM 2.3: аналітика по продуктових лініях (відсортовано)


,productLine,orders_cnt,total_amount,total_profit,quantity,avg_order_value,profit_margin_pct
0,Classic Cars,97,1763136.73,703837.29,16085,18176.667320,39.919609
1,Vintage Cars,88,854551.85,350298.70,10864,9710.816477,40.992094
2,Motorcycles,37,527243.84,222485.41,5976,14249.833514,42.197821
3,Planes,34,471971.46,182273.04,5820,13881.513529,38.619505
4,Trucks and Buses,40,465390.00,182082.20,5024,11634.750000,39.124648
5,Ships,35,337326.10,134731.87,4309,9637.888571,39.941134
6,Trains,22,96285.53,33672.63,1409,4376.615000,34.971641


In [50]:
total_amount_2004 = df["total_amount"].sum()
total_profit_2004 = df["total_profit"].sum()
profit_margin_2004 = (total_profit_2004 / total_amount_2004 * 100) if total_amount_2004 else 0.0

top5_share_pct = (top_countries["total_amount"].sum() / total_amount_2004 * 100) if total_amount_2004 else 0.0

unique_orders = df["orderNumber"].nunique()
unique_customers = df["customerNumber"].nunique()
period_min = df["orderDate"].min().date()
period_max = df["orderDate"].max().date()

top_country_row = gb_country.sort_values("total_amount", ascending=False).head(1)
top_country = top_country_row.index[0]
top_country_revenue = float(top_country_row["total_amount"].iloc[0])

top_pl_row = gb_pl.sort_values("total_amount", ascending=False).head(1)
top_product_line = top_pl_row.index[0]
top_pl_revenue = float(top_pl_row["total_amount"].iloc[0])

summary = pd.DataFrame({
    "Показник": [
        "Загальний дохід, $",
        "Загальний прибуток, $",
        "Маржа прибутку, %",
        "Частка доходу топ-5 країн, %",
        "К-сть унікальних замовлень",
        "К-сть унікальних клієнтів",
        "Початок періоду",
        "Кінець періоду",
        "Найприбутковіша країна",
        "Дохід найприбутковішої країни, $",
        "Найприбутковіша продуктова лінія",
        "Дохід найприбутковішої лінії, $",
    ],
    "Значення": [
        round(total_amount_2004, 2),
        round(total_profit_2004, 2),
        round(profit_margin_2004, 2),
        round(top5_share_pct, 2),
        unique_orders,
        unique_customers,
        str(period_min),
        str(period_max),
        top_country,
        round(top_country_revenue, 2),
        top_product_line,
        round(top_pl_revenue, 2),
    ]
})
print("TRANSFORM 2.4: executive summary")
display(summary)


TRANSFORM 2.4: executive summary


,Показник,Значення
0,"Загальний дохід, $",4515905.51
1,"Загальний прибуток, $",1809381.14
2,"Маржа прибутку, %",40.07
3,"Частка доходу топ-5 країн, %",65.21
4,К-сть унікальних замовлень,151
5,К-сть унікальних клієнтів,89
6,Початок періоду,2004-01-02
7,Кінець періоду,2004-12-17
8,Найприбутковіша країна,USA
9,"Дохід найприбутковішої країни, $",1526499.65


In [52]:
!pip install xlsxwriter


  Using cached xlsxwriter-3.2.5-py3-none-any.whl.metadata (2.7 kB)
Using cached xlsxwriter-3.2.5-py3-none-any.whl (172 kB)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
out_path = "etl_2004_sales.xlsx"

with pd.ExcelWriter(out_path, engine="xlsxwriter") as writer:
    summary.to_excel(writer, index=False, sheet_name="Summary")
    top_countries.to_excel(writer, index=False, sheet_name="Top Countries")
    product_lines.to_excel(writer, index=False, sheet_name="Product Lines")

print(f"LOAD: Excel збережено → {out_path}")


LOAD: Excel збережено → etl_2004_sales.xlsx
